In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20})
import os
import seaborn as sns

In [ ]:
q_path = './Dataset_Question1/'
pics = 10  # pics per subject
other_pics = 140

**SVD FUNCTIONS**

In [ ]:
def to3d(a):
    a_exp = np.expand_dims(a, axis=2)
    return np.repeat(a_exp, repeats=3, axis=2)

def present(a):
    return np.rint(np.clip(a, 0, 255)).astype('int')


def svd_sum(x, k):  # Sum first k terms out of 64
    u, s, vt = np.linalg.svd(x[:, :, 0])
    comps = []
    sums = []
    vals = []
    
    cu_sum = np.zeros((64, 64))
    for i in range(k):
        ui = np.expand_dims(u[:, i], axis=1)
        si = s[i]
        vti = np.expand_dims(vt[i, :], axis=0)
        
        xi = si*np.dot(ui, vti)
        cu_sum = cu_sum + xi
        
        sums.append(cu_sum)
        comps.append(xi)
        vals.append(si)
        
    return comps, sums, vals

**PCA FUNCTIONS**

In [ ]:
def eigen(a):
    vals, vecs = np.linalg.eig(a)
    sort_indices = np.argsort(vals)[::-1]
    return vals[sort_indices], vecs[:, sort_indices]

def pca_eigen(X, p):
    """PCA through eigenvectors of covariance matrix"""
    m, n = X.shape
    mu = np.mean(X, axis=0)
    X_c = X - mu  # mean centering
    C = np.dot(X_c.T, X_c)/m  # covariance matrix: n X n
    
    l, V = eigen(C)
    W = V[:, :p]  # p principal directions: n X p
    
    Z_c = np.dot(X_c, W)  # all projections: m X p
    X_c_approx = np.dot(Z_c, W.T)  # all reconstructions: m X n
    return W, Z_c, X_c_approx

def pca_svd(X, p):
    """PCA through SVD of centered data matrix, FASTER"""
    m, n = X.shape
    
    mu = np.expand_dims(np.mean(X, axis=0), axis=0)
    X_c = X - mu  # mean centering
    
    U, s, Vt = np.linalg.svd(X_c)
    S = np.diag(s)
    V = Vt.T
    
    W = V[:, :p]  # p principal directions: n X p
    
    Z_c = np.dot(U[:, :p], S[:p, :p])  # all projections: m X p
    X_c_approx = np.dot(Z_c, W.T)  # all reconstructions: m X n
    return W, Z_c, X_c_approx

# X = np.random.rand(2000, 2000)
# W, Z_c, X_c_approx = pca_svd(X, p=5)

**TEST RECONSTRUCTIONS**

In [ ]:
test_subject = 6
test_subject_path = os.path.join(q_path, str(test_subject))

K_test = 10
test_subject_image = cv2.imread(test_subject_path+'/1.pgm')
c, s, v = svd_sum(test_subject_image, k=K_test)

plt.figure(figsize=(10, 10))
plt.subplot(121)
plt.title('Original')
plt.imshow(test_subject_image)
plt.subplot(122)
plt.title('Reconstruction: {} terms'.format(K_test))
plt.imshow(to3d(present(s[-1])))

In [ ]:
np.linalg.matrix_rank(test_subject_image[:, :, 0])

**MEAN IMAGE**

In [ ]:
def mean_image(subject):
    M = np.zeros((64, 64, 3))
    subject_path = os.path.join(q_path, str(subject))
    
    for image in os.listdir(subject_path):
        subject_image_path = os.path.join(subject_path, image)
        x = cv2.imread(subject_image_path)
        M += x
    M /= pics
    return M

In [ ]:
plt.figure(figsize=(5, 10))
plt.imshow(present(mean_image(test_subject)))

**Projections onto 1st PD**

In [ ]:
def eigen1(subject):
    
    X = np.zeros((4096, 10))
    subject_path = os.path.join(q_path, str(subject))
    
    i = 0
    for image in os.listdir(subject_path):
        subject_image_path = os.path.join(subject_path, image)
        x = cv2.imread(subject_image_path)  # 64x64x3
        x_vec = np.reshape(x[:, :, 0], (4096))
        X[:, i] = x_vec
        i += 1
    E = pca_svd(X, 1)[1]  # 4096x1
    return E

**MEAN K SVD-TERMS**

In [ ]:
def svd_mean(K, subject):
    all_sums = []
    subject_path = os.path.join(q_path, str(subject))
    
    for image in os.listdir(subject_path):
        subject_image_path = os.path.join(subject_path, image)
        x = cv2.imread(subject_image_path)
        comps, sums, coeffs = svd_sum(x, K)

        all_sums.append(sums)
    all_sums = np.array(all_sums)

    y = np.sum(all_sums[:, -1, :, :], axis=0)/pics  # Average over all 10 pictures of the person
    return to3d(y)

In [ ]:
plt.imshow(present(svd_mean(6, test_subject)))

**EVALUATE ERRORS**

1. With same subject

In [ ]:
m_avg_errors_same = np.zeros(64)
m_max_errors_same = np.zeros(64)
svd_avg_errors_same = np.zeros(64)
svd_max_errors_same = np.zeros(64)

M = mean_image(test_subject)

for k in range(64):
    M_svd = svd_mean(k+1, test_subject)
    
    for image in os.listdir(test_subject_path):
        image_path = os.path.join(test_subject_path, image)
        x = cv2.imread(image_path)
        
        m_avg_errors_same[k] += np.linalg.norm(x[:, :, 0]-M[:, :, 0])/pics
        m_max_errors_same[k] = max(np.linalg.norm(x[:, :, 0]-M[:, :, 0]), m_max_errors_same[k])
                
        svd_avg_errors_same[k] += np.linalg.norm(x[:, :, 0]-M_svd[:, :, 0])/pics
        svd_max_errors_same[k] = max(np.linalg.norm(x[:, :, 0]-M_svd[:, :, 0]), svd_max_errors_same[k])
        
    print('{} term(s) done'.format(k+1))

2. With other subjects

In [ ]:
m_avg_errors_other = np.zeros(64)
m_max_errors_other = np.zeros(64)
svd_avg_errors_other = np.zeros(64)
svd_max_errors_other = np.zeros(64)

M = mean_image(test_subject)

for k in range(64):
    M_svd = svd_mean(k+1, test_subject) 
    
    for other_subject in range(1, 16):
        if other_subject == test_subject:
            continue
            
        other_subject_path = os.path.join(q_path, str(other_subject))
        for image in os.listdir(other_subject_path):
            image_path = os.path.join(other_subject_path, image)
            x = cv2.imread(image_path)
            
            m_avg_errors_other[k] += np.linalg.norm(x[:, :, 0]-M[:, :, 0])/other_pics
            m_max_errors_other[k] = max(np.linalg.norm(x[:, :, 0]-M[:, :, 0]), m_max_errors_other[k])
  
            svd_avg_errors_other[k] += np.linalg.norm(x[:, :, 0]-M_svd[:, :, 0])/other_pics
            svd_max_errors_other[k] = max(np.linalg.norm(x[:, :, 0]-M_svd[:, :, 0]), svd_max_errors_other[k])
        
    print('{} term(s) done'.format(k+1))

In [ ]:
############################## PLOT ERRORS
sns.set_style('darkgrid')
plt.figure(figsize=(15, 10))

sns.lineplot(x=np.arange(64)+1, y=m_avg_errors_same, label='Mean Image: Avg Norm', color='Black')
sns.lineplot(x=np.arange(64)+1, y=m_max_errors_same, label='Mean Image: Max Norm', color='Black')


sns.scatterplot(x=np.arange(64)+1, y=svd_avg_errors_same, label='SVD: Avg Norm', color='Blue')
sns.scatterplot(x=np.arange(64)+1, y=svd_max_errors_same, label='SVD: Max Norm', color='Red')

plt.title('Subject {}: Errors with same subject'.format(test_subject))
plt.xlabel('Terms chosen per example of a subject')
plt.ylabel('Norm over all 10 examples')

In [ ]:
############################## PLOT ERRORS
sns.set_style('darkgrid')
plt.figure(figsize=(15, 10))

sns.lineplot(x=np.arange(64)+1, y=m_avg_errors_other, label='Mean Image: Avg Norm', color='Black')
sns.lineplot(x=np.arange(64)+1, y=m_max_errors_other, label='Mean Image: Max Norm', color='Black')


sns.scatterplot(x=np.arange(64)+1, y=svd_avg_errors_other, label='SVD: Avg Norm', color='Blue')
sns.scatterplot(x=np.arange(64)+1, y=svd_max_errors_other, label='SVD: Max Norm', color='Red')

plt.title('Subject {}: Errors with all other subjects'.format(test_subject))
plt.xlabel('Terms chosen per example of a subject')
plt.ylabel('Norm over all 140 examples')

3. Combined error

In [ ]:
plt.figure(figsize=(15, 10))

sns.lineplot(x=np.arange(64)+1, y=m_avg_errors_same-m_avg_errors_other, label='Mean Image: Avg Norm', color='Black')
sns.scatterplot(x=np.arange(64)+1, y=svd_avg_errors_same-svd_avg_errors_other, label='SVD: Avg Norm', color='Blue')

plt.title('Subject {}: Combined Errors'.format(test_subject))
plt.xlabel('Terms chosen per example of a subject')
plt.ylabel('Error over all examples')

**ACCURACY**

1. Mean

In [ ]:
mean_representations = []
for subject in range(1, 16):
    M = mean_image(subject)
    mean_representations.append(M)
    
    
correct_by_mean = 0
for new_subject in range(1, 16):
    correct_class = new_subject
    
    new_subject_path = os.path.join(q_path, str(new_subject))
    for image in os.listdir(new_subject_path):
        new_subject_image_path = os.path.join(new_subject_path, image)
        x = cv2.imread(new_subject_image_path)  # One of the 150 images
        
        norms = [np.linalg.norm(x[:, :, 0]-mr[:, :, 0]) for mr in mean_representations]
        predicted_class = 1+np.argmin(norms)
        
        if predicted_class == correct_class:
            correct_by_mean += 1
        else:
            print('Wrongly classified')
            print('Image: {}-{}, Predicted Class: {}\n'.format(new_subject, image, predicted_class))
print('Number of examples correctly classified:', correct_by_mean)

In [ ]:
sns.set_style('dark')

plt.figure(figsize=(10, 10))

plt.subplot(1, 2, 1)
plt.axis("off")
plt.imshow(cv2.imread(q_path+'9/9.pgm'))
plt.title('Test Image')

plt.subplot(1, 2, 2)
plt.axis("off")
plt.imshow(cv2.imread(q_path+'8/6.pgm'))
plt.title('Predicted Class')

2. PCA

In [ ]:
pca_representations = []
for subject in range(1, 16):
    
    e1 = eigen1(subject)  # 4096x1
    m = np.reshape(mean_image(subject)[:, :, 0], (4096, 1))
    if np.dot(e1.T, m) < 0:
        e1 *= -1
    
    
    E = to3d(np.reshape(e1, (64, 64)))
    
    smallest, largest = np.min(E), np.max(E)
    E_rescaled = (E-smallest)*255/(largest-smallest)  # Linear scaling to [0-255] range

    pca_representations.append(E_rescaled)
    
correct_by_pca = 0
for new_subject in range(1, 16):
    correct_class = new_subject
    
    new_subject_path = os.path.join(q_path, str(new_subject))
    for image in os.listdir(new_subject_path):
        new_subject_image_path = os.path.join(new_subject_path, image)
        x = cv2.imread(new_subject_image_path)  # One of the 150 images
        
        norms = [np.linalg.norm(x[:, :, 0]-pr[:, :, 0]) for pr in pca_representations]
        predicted_class = 1+np.argmin(norms)
        
        if predicted_class == correct_class:
            correct_by_pca += 1
        else:
            print('Wrongly classified')
            print('Image: {}-{}, Predicted Class: {}\n'.format(new_subject, image, predicted_class))
print('Number of examples correctly classified:', correct_by_pca)

In [ ]:
sns.set_style('dark')

plt.figure(figsize=(20, 10))
for i in range(15):
    plt.subplot(3, 5, i+1)
    plt.axis("off")
    plt.imshow(present(pca_representations[i]))
    plt.title('Subject {}'.format(i+1))

3. SVD

In [ ]:
for k in range(64):
    print('{} term(s)\n'.format(k+1))
    my_representations = []
    for subject in range(1, 16):        
        M = mean_image(subject)
        M_svd = to3d(svd_sum(M, k+1)[1][-1])  # Represent mean with a rank (k+1) matrix
        my_representations.append(M_svd)

        
    correct_by_svd = 0
    for new_subject in range(1, 16):
        correct_class = new_subject

        new_subject_path = os.path.join(q_path, str(new_subject))
        for image in os.listdir(new_subject_path):
            new_subject_image_path = os.path.join(new_subject_path, image)
            x = cv2.imread(new_subject_image_path)

            norms = [np.linalg.norm(x[:, :, 0]-mr[:, :, 0]) for mr in my_representations]
            predicted_class = 1+np.argmin(norms)

            if predicted_class == correct_class:
                correct_by_svd += 1
            else:
                print('Wrongly classified')
                print('Image: {}-{}, Predicted Class: {}\n'.format(new_subject, image, predicted_class))

    print('Number of examples correctly classified: {}\n============\n'.format(correct_by_svd))

In [ ]:
my_representations = []
for subject in range(1, 16):        
    M = mean_image(subject)
    M_svd = to3d(svd_sum(M, k=6)[1][-1])  # Represent mean with a rank 6 matrix
    my_representations.append(M_svd)

sns.set_style('dark')
plt.figure(figsize=(20, 10))
for i in range(15):
    plt.subplot(3, 5, i+1)
    plt.axis("off")
    plt.imshow(present(my_representations[i]))
    plt.title('Subject {}'.format(i+1))